<a href="https://colab.research.google.com/github/odu-cs-580-f21/assignment_5/blob/master/README.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 480/580 - Assignment_5 - Dániel B. Papp

# Notice

Since I wasn't able to use the dataset provided in the assignment document, I used the same dataset but provided by Kaggle in a `.csv` format. The data and the Kaggle challenge can be found [here](https://www.kaggle.com/c/digit-recognizer/data).

# How to run

```console
python3 app.py
```

# How it works

On a high level, the program reads the `.csv` dataset and makes predictions for each image. The predictions are then compared to the actual labels and the accuracy is calculated. The accuracy is then printed to the terminal.

All the necessary mathematical operations will be explained function by function below.

# Neural Network (NN) structure overview

Our NN will have three layers in total. To further improve the accuracy of our model, we could increase the number or layers or the number of nodes in the hidden layer.

1. Input layer
   - The input layer has 784 nodes, which corresponds with the 28x28 pixels of the image. Each individual pixel is normalized to a value between 0 and 1. Since we only care about the pixel either being `rgb(0,0,0)` or `rgb(255,255,255)`, we can normalize each pixel to represent true or false (1 or 0) weather it is black or white.
2. Hidden layer
   - For the sake of simplicity, we will use a single hidden layer with 10 nodes. The value of each node is calculated based on weights and biases applied to the value of the 784 nodes in the input layer.
3. Output layer
   - The output layer has 10 nodes, each representing a digit from 0 through 9. The value of each node is calculated based on weights and biases applied to the value of the 10 nodes in the hidden layer using a softmax activation algorithm.

# The mathematical operations and functions

## Our data

Each of our training example can be represented as a vector of 784 values. These vectors are then stacked into a matrix so we can calculate error from all examples at once with matrix operations.

Our matrix will have an $m {\times} n$ dimension, where m is the number of training examples and n is the number of nodes in the input layer (784). We transpose the matrix so the dimensions will be $n {\times} m$, with each column corresponding to a training example and each row is a node.

## Weight and biases

Between every two layers is a set of connections between every node in the previous layer and every node in the following layer. Which means there is a weight of of $n^{[l]}$ for every `i` in the number of nodes in the previous layer and every `j` in the number of nodes in the following layer.

From this we can conclude that our weights as a matrix will be $n^{[l]}{\times}n^{[l - 1]}$ where $n^{[l]}$ is the number of nodes in the previous layer and $n^{[l - 1]}$ is the number of nodes in the following layer. We call this matrix the $W^{[l]}$ matrix corresponding to layer ${l}$ of our network.

- $W^{[1]}$ will be represneting a $10 {\times} 784$ matrix, taking 784 nodes from the input layer corresponding to 10 nodes in the hidden layer.
- $W^{[2]}$ will have the dimensions of $10{\times}10$

## Forward propagation

The first part of the function that is needed to calculate the forward propogation is the unactived values of the nodes in the first hidden layer. We can calculate that by applying $W^{[1]}$ and $b^{[1]}$ to our input layer. 

This produces $Z^{[1]} = W^{[1]} X + b^{[1]}$ where $X$ has the dimentions of $784{\times}m$, and $W^{[1]}$ has the dimentions of $10{\times}784$.

$W^{[1]} X$ is the dot product between the two, yielding a new matrix of dimensions $10{\times}m$. 

Our bias term $b^{[1]}$ has dimensions of $10{\times}1$, but our goal is to apply the same column of bias to all $m$ columns of traning data. This means that $b^{[1]}$ is effectivly $10{\times}m$ when calculating $Z^{[1]}$. 

$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{softmax}}(Z^{[2]})$$